In [ ]:
import pandas as pd
import sklearn
import numpy as np
import datetime

In [ ]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [ ]:
df.head()

## Q1

In [ ]:
len(df.columns)

## Q2

In [ ]:
df["duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]) 
df["duration_min"] = [x.total_seconds()/60 for x in df["duration"]]
np.std(df["duration_min"])

## Q3

In [ ]:
df_filtered = df[(df.duration_min >= 1) & (df.duration_min <= 60)].reset_index(drop = True)

In [ ]:
df_filtered.shape[0]/df.shape[0] * 100

## Q4

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
df_filtered['PULocationID'] = df_filtered['PULocationID'].astype(str)
df_filtered['DOLocationID'] = df_filtered['DOLocationID'].astype(str)
selected_columns = ["PULocationID", "DOLocationID"]
train_dicts = df_filtered[selected_columns].to_dict(orient='records')

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

## Q5

In [ ]:
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
y_train = df_filtered["duration_min"]

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_train)
root_mean_squared_error(y_train, y_pred)

## Q6

In [ ]:
df_valid = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [ ]:
def prep_val_df(df, dv):
    df["duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]) 
    df["duration_min"] = [x.total_seconds()/60 for x in df["duration"]]
    df = df[(df.duration_min >= 1) & (df.duration_min <= 60)].reset_index(drop = True)
    df['PULocationID'] = df['PULocationID'].astype(str)
    df['DOLocationID'] = df['DOLocationID'].astype(str)
    selected_columns = ["PULocationID", "DOLocationID"]
    train_dicts = df[selected_columns].to_dict(orient='records')
    X_train = dv.transform(train_dicts)
    return X_train

In [ ]:
def get_y(df):
    return df["duration_min"] 

In [ ]:
X_val = prep_val_df(df_valid, dv)

In [ ]:
y_val = get_y(df_valid)

In [ ]:
y_pred = model.predict(X_val)
root_mean_squared_error(y_val, y_pred)